# Model Notebook 

## Business & Data Understanding
#### Revisiting our end goals with sombe EDA knowledge
- Stakeholder is a real estate agency.
- We want to create a tool for a real estate agency to estimate sales or purchase prices given housing info.
- This can be done with a regression model.

## Loading packages, libraries, functions and variables from the EDA notebook.

In [1]:
#Loading the needed packages, libraries, functions and variables from the EDA notebook.
import pandas as pd
from pandas.api.types import is_numeric_dtype
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import statsmodels.api as sm

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
#Original DataFrame
%store -r df_original

In [3]:
#Cleaned DataFrame — from the EDA notebook
%store -r df_clean

In [4]:
from haversine import haversine, Unit

def get_dist(lat, long):
    seattle = (47.608013, -122.335167)
    house = (lat, long)
    return haversine(seattle, house, unit=Unit.MILES)

# Add distance_from_seattle column
df_clean['distance'] = df_clean.apply(lambda x: get_dist(x.lat, x.long), axis=1)

In [33]:
def get_dist_red(lat, long):
    redmond = (47.673988, -122.121513)
    house = (lat, long)
    return haversine(redmond, house, unit=Unit.MILES)

# Add distance_from_seattle column
df_clean['distance_r'] = df_clean.apply(lambda x: get_dist_red(x.lat, x.long), axis=1)

In [55]:
df_clean.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15', 'month', 'basement',
       'renovated', 'age', 'distance', 'distance_r'],
      dtype='object')

In [53]:
def corr_check(df, threshold):
    '''
    Enter dataframe and threshold for correlation
    Returns table of the highly correlated pairs
    '''
    corr_df = df.corr().abs().stack().reset_index().sort_values(0, ascending=False)
    corr_df['pairs'] = list(zip(corr_df.level_0, corr_df.level_1))
    corr_df.set_index(['pairs'], inplace = True)
    corr_df.drop(columns=['level_1', 'level_0'], inplace = True)
    corr_df.columns = ['cc']
    corr_df = corr_df.drop_duplicates()
    corr_df = corr_df[(corr_df['cc'] > threshold) & (corr_df['cc'] < 1)]
    return corr_df

corr_check(df_clean, .7)

,cc
pairs,
"(yr_renovated, renovated)",0.999968
"(yr_built, age)",0.999873
"(sqft_living, sqft_above)",0.876448
"(lat, distance_r)",0.861560
"(sqft_living, grade)",0.762779
"(sqft_living15, sqft_living)",0.756402
"(grade, sqft_above)",0.756073
"(sqft_living, bathrooms)",0.755758
"(sqft_living15, sqft_above)",0.731767


In [5]:
# For consistent randomness
np.random.seed(42)

## Modeling

In [6]:
lr = LinearRegression()

In [7]:
# Our model needs to have only numeric variables.
# Using this function, we can drop all columns without numeric varibales.
# We will input this function within our next function.
def only_numeric(data):
    '''returns a dataframe with only numeric values'''
    for column in data.columns:
        if is_numeric_dtype(data[column]) == False:
            data = data.drop(column, axis=1)
        else:
            continue
    return data

In [8]:
# This returns our y and X for any data frame. 
# Uses all the numeric columns, need to pass a string as a target variable.
def get_y_X(data, target):
    data = only_numeric(data) # Making data only columns with numeric values.
    y = data[target] 
    X = data.drop(target, axis=1)
    return y, X

In [9]:
# This function will return a train / test split variables for an X and y. 
def my_train_test(ys, Xs):
    X_train, X_test, y_train, y_test = train_test_split(Xs, ys, test_size=.2)

    return X_train, X_test, y_train, y_test

In [10]:
# This prediction function is not in effect, work in progress. 
def prediction(ys, Xs):
    y_hat = lr.predict(X)
    rmse = np.sqrt(mean_squared_error(y, y_hat))
    return rmse, y_hat

In [11]:
# Function to compare R2 values and RMSE values of the train and testing models
def train_test_compare(X_tr, X_te, y_tr, y_te):
    model = lr.fit(X_tr, y_tr) # fit the model
    
    #R2 Scores
    train_score = lr.score(X_tr, y_tr)
    test_score = lr.score(X_te, y_te)
    
    #RMSE
    y_hat_train = lr.predict(X_tr)
    y_hat_test = lr.predict(X_te)
    
    train_rmse = np.sqrt(mean_squared_error(y_tr, y_hat_train))
    test_rmse = np.sqrt(mean_squared_error(y_te, y_hat_test))
    
    #intercept / coef
    inter = model.intercept_
    stats = sm.OLS(y_tr, sm.add_constant(X_tr)).fit()
    summary = stats.summary()
    
    return print(f' training data R2: {train_score}\n testing data R2: {test_score} \
                    \n training data rmse: {train_rmse}\n testing data rmse: {test_rmse} \
                    \n {summary}') 

In [12]:
# I am not sure if I am doing the RMSE correctly, but I am pretty confident with the R2

## Model with Test Data

In [34]:
# Using test data to demonstrate
test_data = df_clean.loc[:,['price', 'bedrooms', 'condition', 'sqft_living']]

In [35]:
y, X = get_y_X(test_data, 'price')

X_train, X_test, y_train, y_test = my_train_test(y, X)

train_test_compare(X_train, X_test, y_train, y_test)

 training data R2: 0.5156306459029397
 testing data R2: 0.5080073861437922                     
 training data rmse: 258083.04765845003
 testing data rmse: 247694.04917739457                     
                             OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.516
Model:                            OLS   Adj. R-squared:                  0.516
Method:                 Least Squares   F-statistic:                     6129.
Date:                Tue, 04 Jan 2022   Prob (F-statistic):               0.00
Time:                        10:32:01   Log-Likelihood:            -2.3980e+05
No. Observations:               17277   AIC:                         4.796e+05
Df Residuals:                   17273   BIC:                         4.796e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
             

## Model on with Entire Clean DF 

In [36]:
y, X = get_y_X(df_clean, 'price')
X_train, X_test, y_train, y_test = my_train_test(y, X)
train_test_compare(X_train, X_test, y_train, y_test)

 training data R2: 0.6966248058437378
 testing data R2: 0.6929332580243823                     
 training data rmse: 203654.5787872035
 testing data rmse: 198180.49150982802                     
                             OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.697
Model:                            OLS   Adj. R-squared:                  0.696
Method:                 Least Squares   F-statistic:                     1887.
Date:                Tue, 04 Jan 2022   Prob (F-statistic):               0.00
Time:                        10:32:07   Log-Likelihood:            -2.3571e+05
No. Observations:               17277   AIC:                         4.715e+05
Df Residuals:                   17255   BIC:                         4.716e+05
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
              

## Model with Dummy Variables for Categories

In [37]:
df_clean_dumm = df_clean.copy()

In [38]:
# Get dummies
zipcode_dummies = pd.get_dummies(df_clean_dumm['zipcode'], drop_first=True)
waterfront_dummies = pd.get_dummies(df_clean_dumm['waterfront'], drop_first=True)
view_dummies = pd.get_dummies(df_clean_dumm['view'], drop_first=True)
month_dummies = pd.get_dummies(df_clean_dumm['month'], drop_first=True)

df_clean_dumm = pd.concat([df_clean_dumm, waterfront_dummies, 
                           view_dummies, month_dummies, zipcode_dummies], axis=1)

In [73]:
df_clean_dumm

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,NO,NONE,...,0,0,0,0,0,0,1,0,0,0
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,0,0,0,0,0,0,0,0,0,0
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,0,0,0,0,0,0,0,0,0,0
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,0,0,0,0,0,0,0,0,0,0
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,2014-05-21,360000.0,3,2.50,1530,1131,3.0,NO,NONE,...,0,0,0,0,0,0,0,0,0,0
21593,6600060120,2015-02-23,400000.0,4,2.50,2310,5813,2.0,NO,NONE,...,1,0,0,0,0,0,0,0,0,0
21594,1523300141,2014-06-23,402101.0,2,0.75,1020,1350,2.0,NO,NONE,...,0,0,0,0,0,0,0,0,0,0
21595,291310100,2015-01-16,400000.0,3,2.50,1600,2388,2.0,NO,NONE,...,0,0,0,0,0,0,0,0,0,0


In [39]:
y, X = get_y_X(df_clean_dumm, 'price')
X_train, X_test, y_train, y_test = my_train_test(y, X)
train_test_compare(X_train, X_test, y_train, y_test)

 training data R2: 0.816467569620108
 testing data R2: 0.8009396726916407                     
 training data rmse: 156213.16782643783
 testing data rmse: 168671.75351668193                     
                             OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.816
Model:                            OLS   Adj. R-squared:                  0.815
Method:                 Least Squares   F-statistic:                     734.5
Date:                Tue, 04 Jan 2022   Prob (F-statistic):               0.00
Time:                        10:32:17   Log-Likelihood:            -2.3113e+05
No. Observations:               17277   AIC:                         4.625e+05
Df Residuals:                   17172   BIC:                         4.633e+05
Df Model:                         104                                         
Covariance Type:            nonrobust                                         
              

## Model with non-Luxury houses w/ Dummy Variabls

In [40]:
non_lux = df_clean_dumm.copy()

In [71]:
non_lux = non_lux[non_lux['price'] < 1000000]
non_lux.drop(columns=['yr_built'], inplace=True)

In [72]:
y, X = get_y_X(non_lux, 'price')
X_train, X_test, y_train, y_test = my_train_test(y, X)
train_test_compare(X_train, X_test, y_train, y_test)

 training data R2: 0.8396062596394325
 testing data R2: 0.8335583803701311                     
 training data rmse: 78549.46330303888
 testing data rmse: 78665.94800613106                     
                             OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.840
Model:                            OLS   Adj. R-squared:                  0.839
Method:                 Least Squares   F-statistic:                     836.7
Date:                Tue, 04 Jan 2022   Prob (F-statistic):               0.00
Time:                        10:46:14   Log-Likelihood:            -2.0413e+05
No. Observations:               16085   AIC:                         4.085e+05
Df Residuals:                   15984   BIC:                         4.092e+05
Df Model:                         100                                         
Covariance Type:            nonrobust                                         
               

In [56]:
model = sm.OLS(y_train, sm.add_constant(X_train)).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.837
Model:                            OLS   Adj. R-squared:                  0.836
Method:                 Least Squares   F-statistic:                     821.2
Date:                Tue, 04 Jan 2022   Prob (F-statistic):               0.00
Time:                        10:40:21   Log-Likelihood:            -2.0416e+05
No. Observations:               16085   AIC:                         4.085e+05
Df Residuals:                   15984   BIC:                         4.093e+05
Df Model:                         100                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
id            -4.648e-08   2.26e-07     -0.205      0.837    -4.9e-07    3.97e-07
bedrooms      -2603.8803    889.951     -2.926      0.003   -4348.285    -859.476
bathrooms      1.161e+04   1549.704      7.490      0.000    8570.025    1.46e+04
sqft_living      62.4153      2.198     28.399      0.000      58.107      66.723
sqft_lot          0.3035      0.017     17.935      0.000       0.270       0.337
floors         -1.52e+04   1848.315     -8.223      0.000   -1.88e+04   -1.16e+04
condition      2.483e+04   1101.723     22.541      0.000    2.27e+04     2.7e+04
grade          4.099e+04   1078.280     38.018      0.000    3.89e+04    4.31e+04
sqft_above       46.2334      2.245     20.594      0.000      41.833      50.634
yr_built       3.557e+04   4223.093      8.424      0.000    2.73e+04    4.39e+04
yr_renovated   2053.6506    226.258      9.077      0.000    1610.159    2497.142
sqft_living15    34.3036      1.816     18.888      0.000      30.744      37.863
basement      -7.182e+07   8.51e+06     -8.439      0.000   -8.85e+07   -5.51e+07
renovated     -4.062e+06   4.51e+05     -8.997      0.000   -4.95e+06   -3.18e+06
age            3.599e+04   4222.700      8.523      0.000    2.77e+04    4.43e+04
distance      -5186.3807    818.606     -6.336      0.000   -6790.941   -3581.820
distance_r    -2323.4325    867.765     -2.677      0.007   -4024.349    -622.516
YES            1.483e+05   1.54e+04      9.640      0.000    1.18e+05    1.78e+05
EXCELLENT      8.946e+04      1e+04      8.924      0.000    6.98e+04    1.09e+05
FAIR           1896.4138   6360.067      0.298      0.766   -1.06e+04    1.44e+04
GOOD            1.83e+04   6086.792      3.006      0.003    6364.783    3.02e+04
NONE          -4.952e+04   3425.353    -14.456      0.000   -5.62e+04   -4.28e+04
2              5563.2973   3900.321      1.426      0.154   -2081.770    1.32e+04
3              2.063e+04   3605.907      5.722      0.000    1.36e+04    2.77e+04
4              2.852e+04   3519.229      8.104      0.000    2.16e+04    3.54e+04
5              2.813e+04   4653.240      6.045      0.000     1.9e+04    3.73e+04
6              3.655e+04   5507.094      6.638      0.000    2.58e+04    4.73e+04
7               3.23e+04   5502.142      5.870      0.000    2.15e+04    4.31e+04
8              3.354e+04   5548.684      6.044      0.000    2.27e+04    4.44e+04
9              2.864e+04   5569.332      5.143      0.000    1.77e+04    3.96e+04
10             3.007e+04   5563.128      5.406      0.000    1.92e+04     4.1e+04
11             3.193e+04   5689.017      5.612      0.000    2.08e+04    4.31e+04
12             3.075e+04   5670.831      5.422      0.000    1.96e+04    4.19e+04
98002          6487.5456   7807.846      0.831      0.406   -8816.711    2.18e+04
98003         -5888.7617   7108.539     -0.828   

## Model with non-Luxury and non-Cheap houses w/ Dummy Variabls

In [57]:
no_lux_cheap = non_lux.copy()

In [58]:
no_lux_cheap = no_lux_cheap[no_lux_cheap['price'] > 100000]

In [59]:
y, X = get_y_X(no_lux_cheap, 'price')
X_train, X_test, y_train, y_test = my_train_test(y, X)
train_test_compare(X_train, X_test, y_train, y_test)

 training data R2: 0.839664151955087
 testing data R2: 0.8336827370921276                     
 training data rmse: 78133.07842334177
 testing data rmse: 79518.453739375                     
                             OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.840
Model:                            OLS   Adj. R-squared:                  0.839
Method:                 Least Squares   F-statistic:                     803.5
Date:                Tue, 04 Jan 2022   Prob (F-statistic):               0.00
Time:                        10:40:30   Log-Likelihood:            -2.0374e+05
No. Observations:               16061   AIC:                         4.077e+05
Df Residuals:                   15956   BIC:                         4.085e+05
Df Model:                         104                                         
Covariance Type:            nonrobust                                         
                  

## Model with non-Luxury houses w/ Dummy Variables - drop recurring columns

In [62]:
non_lux_drop = non_lux.copy()
non_lux_drop = non_lux_drop.drop(['id','lat', 'long', 'sqft_lot15', 'month', 'waterfront', 'zipcode', 'view'], axis=1)

In [63]:
y, X = get_y_X(non_lux_drop, 'price')
X_train, X_test, y_train, y_test = my_train_test(y, X)
train_test_compare(X_train, X_test, y_train, y_test)

 training data R2: 0.8374905874385258
 testing data R2: 0.8404717790853624                     
 training data rmse: 78776.42634772856
 testing data rmse: 78211.17930661605                     
                             OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.837
Model:                            OLS   Adj. R-squared:                  0.836
Method:                 Least Squares   F-statistic:                     823.7
Date:                Tue, 04 Jan 2022   Prob (F-statistic):               0.00
Time:                        10:41:28   Log-Likelihood:            -2.0417e+05
No. Observations:               16085   AIC:                         4.085e+05
Df Residuals:                   15984   BIC:                         4.093e+05
Df Model:                         100                                         
Covariance Type:            nonrobust                                         
               

## Visualize Train vs. Test

In [28]:
# I want to make another function to visualize both the training and test, not sure if possible

In [29]:
# plt.scatter(X_test, y_test, color="black")
# plt.plot(X_test, y_pred, color="blue", linewidth=3);

# Other

In [30]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             21597 non-null  int64         
 1   date           21597 non-null  datetime64[ns]
 2   price          21597 non-null  float64       
 3   bedrooms       21597 non-null  int64         
 4   bathrooms      21597 non-null  float64       
 5   sqft_living    21597 non-null  int64         
 6   sqft_lot       21597 non-null  int64         
 7   floors         21597 non-null  float64       
 8   waterfront     21597 non-null  object        
 9   view           21597 non-null  object        
 10  condition      21597 non-null  int64         
 11  grade          21597 non-null  int64         
 12  sqft_above     21597 non-null  int64         
 13  sqft_basement  21597 non-null  object        
 14  yr_built       21597 non-null  int64         
 15  yr_renovated   2159

In [31]:
df_clean.corr().abs()['price'].sort_values()

high_corr_cols = ['sqft_living', 'sqft_above', 'sqft_living15', 'bathrooms', 'sqft_basement', 'bedrooms']

In [32]:
y = df_clean['price']
X = df_clean
    
reg = LinearRegression().fit(X, y)

plt.scatter(X, y, color='green')
plt.plot(X, reg.predict(X))
plt.xlabel('sqft_living')
plt.ylabel('Price');

TypeError: invalid type promotion

In [ ]:
for x in high_corr_cols:
    y = df_clean['price']
    X = df_clean[x]
    
    reg = LinearRegression().fit(X, y)

    plt.scatter(X, y, color='green')
    plt.plot(X, reg.predict(X))
    plt.xlabel(x)
    plt.ylabel('Price');